# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import json
import pandas as pd
from sql_queries import *

In [2]:
%run create_tables.py

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
filepath = "data/song_data"

In [6]:
song_files = get_files(filepath)

In [7]:
df = pd.DataFrame(pd.read_json(song_files[0], lines=True, orient='columns'))
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARD7TVE1187B99BFB1,NaN,California - LA,NaN,Casual,218.93179,1,SOMZWCG12A8C13C480,I Didn't Mean To,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()
song_data[0]

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
for song in song_data:
    cur.execute(song_table_insert, song)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values.tolist()
artist_data[0]

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [12]:
for artist in artist_data:
    cur.execute(artist_table_insert, artist)
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [13]:
filepath = "data/log_data"

In [14]:
log_files = get_files(filepath)

In [15]:
df = pd.DataFrame(pd.read_json(log_files[0], lines=True, orient='columns'))
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Stephen Lynch,Logged In,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",PUT,NextSong,1.540992e+12,829,Jim Henson's Dead,200,1543537327796,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,91


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [16]:
print("Previous dataframe size: ", len(df))
print("Available types in 'page':", df['page'][df['page'].duplicated()==False].tolist())

Previous dataframe size:  388
Available types in 'page': ['NextSong', 'Home', 'Logout', 'Settings', 'Downgrade', 'Login', 'Help', 'Error', 'Upgrade']


In [17]:
df = df[df['page'] == "NextSong"].reset_index()
print("Current dataframe size: ",len(df))

Current dataframe size:  330


In [18]:
time_info = df['ts'].drop_duplicates()
time_info = pd.Series(sorted(pd.to_datetime(time_info,unit='ms')))

In [19]:
# timestamp, hour, day, week of year, month, year, and weekday
time_data = [time_info.dt.strftime('%Y-%m-%d %H:%M:%S'), time_info.dt.hour.tolist(), time_info.dt.day.tolist(), time_info.dt.weekofyear.tolist(), time_info.dt.month.tolist(), time_info.dt.year, time_info.dt.weekday.tolist()]
column_labels = ['timestamp', 'hour', 'day', 'week of year', 'month', 'year', 'weekday']

In [20]:
tmp_dict = dict()
for n in range(len(column_labels)):
    tmp_dict[column_labels[n]] = time_data[n]
time_df = pd.DataFrame(tmp_dict)
time_df.head(5)

,timestamp,hour,day,week of year,month,year,weekday
0,2018-11-30 00:22:07,0,30,48,11,2018,4
1,2018-11-30 01:08:41,1,30,48,11,2018,4
2,2018-11-30 01:12:48,1,30,48,11,2018,4
3,2018-11-30 01:17:05,1,30,48,11,2018,4
4,2018-11-30 01:20:56,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [21]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [22]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
print(user_df.head())
print("Available user levels:", user_df['level'][user_df['level'].duplicated()==False].tolist())

  userId firstName lastName gender level
0     91    Jayden     Bell      M  free
1     73     Jacob    Klein      M  paid
2     73     Jacob    Klein      M  paid
3     73     Jacob    Klein      M  paid
4     73     Jacob    Klein      M  paid
Available user levels: ['free', 'paid']


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [23]:
user_df = user_df.replace("",None)
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [24]:
df = df.replace("",None)
for index, row in df.iterrows():
    # get songid and artistid from song and artist tables
    cur.execute(song_select, ('%'+str(row.song)+'%', '%'+str(row.artist)+'%', float(row.length)))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
        print(results)
    else:
        songid, artistid = None, None

    # insert songplay record
    timestamp = pd.to_datetime(row.ts, unit='ms').strftime('%Y-%m-%d %H:%M:%S')
    songplay_data = (timestamp, row.userId, row.level, str(songid), str(artistid), row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [25]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [26]:
%run etl.py

80 files found in data/song_data
1/80 files processed.
2/80 files processed.
3/80 files processed.
4/80 files processed.
5/80 files processed.
6/80 files processed.
7/80 files processed.
8/80 files processed.
9/80 files processed.
10/80 files processed.
11/80 files processed.
12/80 files processed.
13/80 files processed.
14/80 files processed.
15/80 files processed.
16/80 files processed.
17/80 files processed.
18/80 files processed.
19/80 files processed.
20/80 files processed.
21/80 files processed.
22/80 files processed.
23/80 files processed.
24/80 files processed.
25/80 files processed.
26/80 files processed.
27/80 files processed.
28/80 files processed.
29/80 files processed.
30/80 files processed.
31/80 files processed.
32/80 files processed.
33/80 files processed.
34/80 files processed.
35/80 files processed.
36/80 files processed.
37/80 files processed.
38/80 files processed.
39/80 files processed.
40/80 files processed.
41/80 files processed.
42/80 files processed.
43/80 file